In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import ssl
import os
import re

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [2]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [4]:


def get_html_from_url(url):
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver') 
  browser.set_page_load_timeout(30) 
  browser.get(url)
   
  last_height = browser.execute_script('return document.body.scrollHeight')

  flag=1

  while flag==1:
    
    try:
       browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infine_scroll(last_height))
       last_height = new_height

    except:
        print("End of page reached")
        flag = 0
  html = browser.page_source
  return html

In [5]:
#helper function get first number in string

def str_first_num(s):
    first_num = ''
    flag_digit=False
    for el in s: 
        if el.isdigit():
            first_num += el
            flag_digit = True
        else:
            if flag_digit:
                break
    #If there are no upvotes replace '' with 0
    if first_num == '':
        first_num = 0
    return first_num
    

In [6]:
def unpack_list(li):
    unpacked_list = []
    for el in li:
        if isinstance(el, list):
            unpacked_list.extend([*el])
        elif isinstance(el, str):
            unpacked_list.extend([el])
    return unpacked_list


In [7]:
# a helper function that removes unwanted words/sections like url links
def strip_rgx_words(inp, regex_ = ['image','url']):
    #remove urls or images
    rgx_word = None
    for rgx in regex_:
      if rgx in inp:
          rgx_word = rgx
          break
    if rgx_word is not None:
      out = inp.split(rgx_word)[0]
    else:
      out =inp
    return out

In [8]:
#read question urls from csv file
src_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\quet_indexes.csv"
df = pd.read_csv(src_path)

In [9]:
#get list of urls
url_list = list(df['url'].values)

In [10]:
url_list

['https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/How-long-could-the-brain-survive-theoretically-if-we-had-the-technology-to-replace-all-other-body-parts-as-you-aged-with-functioning-organs-that-are-grown-in-a-lab-using-your-DNA',
 'https://www.quora.com/What-are-the-top-10-emerging-technologies-in-the-next-5-10-years-2020%E2%80%932025',
 'https://www.quora.com/What-are-the-upcoming-emerging-technologies-in-software-industry',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-most-advanced-technologies-that-people-dont-know-about-yet',
 'https://www.quora.com/When-will-Fusion-reactors-become-a-reality',
 'https://www.quora.com/In-future-IOT-Internet-of-things-is-trending-technology-or-not',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-solutions-to-emerging-issues-in-communication']

In [22]:
def fill_list_counts(inf):
    answers_upvotes = []
    for ans_inf in inf:
        if ans_inf == []:
           val = 0
        else:
            val = ans_inf[0].find_next(text=True).strip()
        answers_upvotes.append(val)
    return answers_upvotes

def fill_list_str(inf):
    answers_upvotes = []
    for ans_inf in inf:
        if ans_inf[0] == []:
           val = 'UNKNOWN'
        else:
            val = ans_inf[0].find_next(text=True).strip()
        answers_upvotes.append(val)
    return answers_upvotes


def answers_into_df(url):
    '''input: url for scrapping answers to a particular question.
        out: df with answers for each url/question'''
    #request url html
    
    #create soup object with htmal parser
    page_html = get_html_from_url(url)
    soup = BeautifulSoup(page_html)

    boxes = soup.find_all('div', {'class':"q-box qu-pt--medium qu-hover--bg--darken"})
    answers = [box.find_all('span', {'class':"q-box qu-userSelect--text"}) for box in boxes]
    answers_text = [ans[0].find_next(text=True).strip() for ans in answers] 
    answers_text = unpack_list(answers_text)
    #get a list of answer author upvotes and shares
    #answers_info = [box.find_all('button', {'role':'button'}) for box in boxes]
    answers_upvotes_info = [box.find_all('div', {'class':'q-text qu-overflow--hidden qu-display--inline-flex qu-ml--tiny qu-minHeight--20 qu-color--gray qu-minWidth--20'}) for box in boxes]
    answers_upvotes = []
    
    #answers upvotes:
    answers_upvotes = fill_list_counts(answers_upvotes_info)
    
    names_info = [box.find_all('div', {'class':'q-inlineFlex qu-alignItems--center qu-wordBreak--break-word'}) for box in boxes]
    answers_name = fill_list_str(names_info)
       
    author_counts = [box.find_all('span', {'class':"q-text qu-bold"}) for box in boxes]

    author_answers_count = []
    answer_views = []
        
    for i, aut_co in enumerate(author_counts):
        try:
            ans_co = aut_co[0].find_next(text=True).strip()
            ans_view = aut_co[1].find_next(text=True).strip()
        except:
            print("list is empty")
            ans_co = 0
            ans_view = 0
        author_answers_count.append(ans_co)
        answer_views.append(ans_view)

    #create df with nans to fill it later with values
    columns = ["Answer",  "name", "upvoteCount", "answerCount", "answerViews"]

    x_shape = (len(answers), len(columns))
    x = np.tile(np.nan, x_shape)
   
    answers_df = pd.DataFrame(x, columns = columns)

    answers_df['Answer'] = pd.DataFrame(answers_text)   
    answers_df['name'] = pd.DataFrame(answers_name)
    answers_df['upvoteCount'] = pd.DataFrame(answers_upvotes)
    answers_df['answerCount'] = pd.DataFrame(author_answers_count)
    answers_df['answerViews'] = pd.DataFrame(answer_views)

    return answers_df 

In [23]:
#show one example
answers_into_df(url_list[1])

C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached
list is empty


,Answer,name,upvoteCount,answerCount,answerViews
0,According to my neurobiology professor back in...,Franklin Veaux,2.1K,47K,763.3M
1,Replacing other body parts doesn’t matter. You...,Kurt Guntheroth,9,13.6K,164.1M
2,Identity Over Time,Comet,9,5.2K,9.4M
3,Is it possible to keep a human brain alive wit...,Michael Zions,999,141,1.8M
4,Are you of the mistaken opinion that the brain...,Bill Bigos,0,1.1K,999.1K
5,Who can help me? I am a full stack developer w...,Carolina Silva,2.8K,0,0
6,How does a human brain work?,Faisal Hourani,999,1.1K,1.9M
7,Is it true that the brain works worse with age...,Claude Lambert,99,9.6K,6.7M
8,How long can you live without a brain?,Per Lyngemark,999,5.3K,4.8M
9,Can the brain still survive outside of the bod...,Jens Mowatt,9,569,2.3M


In [24]:
columns = ["question", "Answer", "name", "upvoteCount", "answerCount", "answerViews"]
df = pd.DataFrame(data = [], columns = columns)

for i, url in enumerate(url_list):
    temp_df = answers_into_df(url)
    temp_df["question"] = url
    #rearrange columns order to match main df columns order
    temp_df = temp_df[columns]

    df = df.append(temp_df)


C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty


C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\1667476013.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached


C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\1667476013.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


...

C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\1667476013.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty
list is empty


C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\1667476013.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached
list is empty
list is empty
list is empty
list is empty
list is empty


C:\Users\Ravit\AppData\Local\Temp\ipykernel_10816\1667476013.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)


In [26]:
len(df)

636

In [27]:
#save df to csv
save_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info"
final_path = os.path.join(save_path, "ET_RCS_22SEP15_quora_data.csv")
df.to_csv(final_path)